# Hello!

Welcome to my "Google Research - Identify Contrails to Reduce Global Warming" submission. Here's the rundown:
* UNET for image segmentation
* A lot of stuff I copied from other people sry

# Imports

In [23]:
import albumentations as A
from albumentations.pytorch import ToTensorV2

import numpy as np
import os
import pandas as pd

from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms.functional as TF
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

# Dataset

In [24]:
class ContrailsDataset(torch.utils.data.Dataset):
    def __init__(self, df, train=True):
        
        # Pass in dataframe (3 columns: record_id, train, path)
        # Note: ContrailsDataset object doesn't store images. Only stores path 
        # to images (from 'df'). Each time ContrailsDataset object is indexed 
        # into, it retrieves the sample and label corresponding to the index. 
        self.df = df
        self.trn = train
    
    # Handles indexing into ContrailsDataset object (i.e. '[]' operation)
    def __getitem__(self, index):
        
        # Accesses sample and label here (according to index)
        row = self.df.iloc[index]
        con_path = row.path
        con = np.load(str(con_path))
        
        # Selects all dimensions before last one. In last dimension, selects all
        # elements, excluding last one (':-1' means slice to but not include last 
        # one).
        # All dimensions excluding last element of last one makes up the sample image.
        img = con[..., :-1]
        
        # Selects all dimensions before last one + last element of last dimension
        # All dimensions + last element of last one make up label
        label = con[..., -1]
        
        img = torch.tensor(img)
        label = torch.tensor(label)
        
        img = img.permute(2, 0, 1)
        
        display("indexing into thing")
#         display(img.shape)
            
        # Returns tuple !!! (sample, label)
        # Indexing into ContrailDataset returns both sample and label!
        return img.float(), label.float()
    
    def __len__(self):
        return len(self.df)

# Model

In [25]:
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DoubleConv, self).__init__()
        self.conv = nn.Sequential(
            
            # in channels, out channels, kernel size, stride, padding
            # padding = 1 --> input height and width is same before and after convolution
            nn.Conv2d(in_channels, out_channels, 3, 1, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, 3, 1, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )
        
    def forward(self, x):
        return self.conv(x)
    

class UNET(nn.Module):
    def __init__(
        
        # features are the numbers at the top of the blocks
        self, in_channels=3, out_channels=1, features=[64, 128, 256, 512] 
    ):
        super(UNET, self).__init__();
        
        # store convolution layers, since we want to do batch eval
        self.ups = nn.ModuleList()
        self.downs = nn.ModuleList()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        # 161 x 161 --> 80 x 80 --> output: 160x160
        # Input must be divisible by 16, since it is being divided by 2 four times (however we generalized it later on)
        
        
        # Down part of UNET (where "U" goes down)
        for feature in features:
            
            # Append layer into down
            self.downs.append(DoubleConv(in_channels, feature))
            in_channels = feature
            
        # Up part of UNET (where "U" goes up)
        # We used transpose convolutions
        for feature in reversed(features):
            self.ups.append(
                nn.Conv2d(
                    feature*2, feature, kernel_size=2, stride=2
                )
            )
            
            # Append double conv (where U transition from down to up)
            self.ups.append(DoubleConv(feature*2, feature))
            
            self.bottleneck = DoubleConv(features[-1], features[-1]*2)
            self.final_conv = nn.Conv2d(features[0], out_channels, kernel_size=1)
            
    def forward(self, x):
        
#         display('x:')
#         display(x.shape)
#         display(x)
        
        # Store skipped connections here
        # Note that we are adding back the skipped connections in the reverse order we are adding into this list
        skip_connections = []
        
        # Iterate through each convolutional layer in downs
        for down in self.downs:
            
            # Apply convolutional layer on x
            x = down(x)
            skip_connections.append(x)
            x = self.pool(x)
            
        x = self.bottleneck(x)
        skip_connections = skip_connections[::-1]
        
        for idx in range(0, len(self.ups), 2):
            x = self.ups[idx](x)
            skip_connection = skip_connections[idx//2]
            
            # Generalizes when the width/height of layer isn't divisible by 2
            if x.shape != skip_connection.shape:
                
                # "skip_connection.shape[2:]" gets the height and width (skipping batch size and channels)
                x = TF.resize(x, size=skip_connection.shape[2:])
                
            
            # Concatenate skip connections
            concat_skip = torch.cat((skip_connection, x), dim=1)
            x = self.ups[idx+1](concat_skip)
        
        return self.final_conv(x)

# Utility Functions

In [26]:
def save_checkpoint(state, filename="my_checkpoint.pth.tar"):
    print("=> Saving checkpoint")
    torch.save(state, filename)

In [27]:
def load_checkpoint(checkpoint, model):
    print("=> Loading checkpoint")
    model.load_state_dict(checkpoint["state_dict"])

In [28]:
def get_loaders(
    train_df,
    valid_df,
    batch_size,
    train_transform,
    val_transform,
    num_workers=4,
    pin_memory=True
):
    
    # Get Contrails Dataset (class: ContrailsDataset, inherited: torch.data.Dataset)
    train_ds = ContrailsDataset(
        train_df,
        train=True
    )

    # Wraps Dataset using DataLoader, which makes it iterable
    #   'train_ds' is the pytorch.Dataset object
    #   'batch_size' is the size of the "mini-batches" we want to load the data in
    #   'pin_memory' "For data loading, passing pin_memory=True to a DataLoader will automatically put the fetched data Tensors in pinned memory, and thus enables faster data transfer to CUDA-enabled GPUs."
    #   'shuffle' will shuffle the data before every epoch
    train_loader = DataLoader(
        train_ds,
        batch_size=batch_size,
        num_workers=num_workers,
        pin_memory=pin_memory,
        shuffle=True
    )

    valid_ds = ContrailsDataset(
        valid_df,
        train=False
    )

    valid_loader = DataLoader(
        valid_ds,
        batch_size=batch_size,
        num_workers=num_workers,
        pin_memory=pin_memory,
        shuffle=False
    )

    return train_loader, valid_loader

In [29]:
def check_accuracy(loader, model, device="cuda"):
    num_correct = 0
    num_pixels = 0
    dice_score = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            print(f"x.shape {x.shape}, y.shape {y.shape}")
            
            x = x.to(device)
            
            # Label doesn't have channel because grayscale, so must .unsqueeze(1)
            y = y.to(device).unsqueeze(1)

            # This is for binary
            preds = torch.sigmoid(model(x))
            preds = (preds > 0.5).float()
            num_correct += (preds == y).sum()
            num_pixels += torch.numel(preds)

            # Dice score (at least in this case) is a better metric for evaluating accuracy of segmentation (as if model outputs all black pixels it can get 80% accuracy)
            dice_score += (2*(preds*y).sum())/(
                (preds+y).sum() + 1e-8
            )

    # Display accuracy
    print(
        f"Got {num_correct}/{num_pixels} with acc {num_correct/num_pixels*100:.2f}"
    )
    print(f"Dice score: {dice_score/len(loader)}")
    
    model.train()

In [30]:
def save_predictions_as_imgs(
    loader, model, folder="saved_images/", device="cuda"
):
    model.eval()
    
    # Iterates over each batch, NOT (unless BATCH_SIZE=1,) EACH SAMPLE
    for idx, (x, y) in enumerate(loader):
        
        # Moves "x" to the specified device (usually for sake of efficiency, e.g. when training a model)
        x = x.to(device=device)
        with torch.no_grad():
            preds = torch.sigmoid(model(x))
            preds = (preds > 0.5).float()
            
        # torchvision.utils.save_image (https://pytorch.org/vision/stable/generated/torchvision.utils.save_image.html):
        #   Arguments:
        #       Tensor
        #           The Tensor to be saved as an image.
        #       String/File
        #           Location to save image
        #   "If given a mini-batch tensor, saves the tensor as a grid of images by calling make_grid."
        #       I.e. this situation
        
        # torchvision.utils.make_grid (https://pytorch.org/vision/stable/generated/torchvision.utils.make_grid.html):
        #   Arguments:
        #       Tensor
        #           "4D mini-batch Tensor of shape (B x C x H x W) or a list of images all of the same size."
        #   Returns:
        #       Tensor
        #           Contains grid of images
        #   Note: One of the arguments is "nrow," which specifies the number of images in each row. The default is 8, 
        #       thus the predictions have 8 images per row.
         
        #   Note: We pass in a 4D tensor (preds), of shape [B, C, H, W] ("mini batch" size, channels, height, width).
        #       Thus, torchvision.utils.make_grid is called and we get a total of 16 images per grid, with 8 images 
        #       per row. If we want the single image predictions, we can set BATCH_SIZE = 1, or just have no batches
        #       at all.
        torchvision.utils.save_image(
            preds, f"{folder}/pred_{idx}.png"
        )
        torchvision.utils.save_image(y.unsqueeze(1), f"{folder}/{idx}.png")
        
    model.train()

# Train Model

### Hyperparameters and more

In [31]:
# Hyperparameters etc.
LEARNING_RATE = 1e-4
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 16
NUM_EPOCHS = 1
NUM_WORKERS = 2
IMAGE_HEIGHT = 160
IMAGE_WIDTH = 240
PIN_MEMORY = True

# Set to true after finished training model
LOAD_MODEL = True

# TRAIN_IMG_DIR = "Data/train_images/"
# TRAIN_MASK_DIR = "Data/train_masks/"
# VAL_IMG_DIR = "Data/valid_images/"
# VAL_MASK_DIR = "Data/valid_masks/"

ROOT = os.path.join(os.getcwd(), "MachineLearning", "Unet", "Data")
ROOT2 = os.path.join(os.getcwd(), "MachineLearning", "Unet")

TRAIN_IMG_DIR = os.path.join(ROOT, "train_images")
TRAIN_MASK_DIR = os.path.join(ROOT, "train_masks")
VAL_IMG_DIR = os.path.join(ROOT, "valid_images")
VAL_MASK_DIR = os.path.join(ROOT, "valid_masks")

### Train Function
The train function for one epoch

In [32]:
def train_fn(loader, model, optimizer, loss_fn, scaler):
    # For progress bar
    loop = tqdm(loader)
    
    # Iterates over each batch
    for batch_idx, (data, targets) in enumerate(loop):
#         display('data:')
#         display(data.shape)
#         display(data)
        
#         display('targets:')
#         display(targets.shape)
#         display(targets)
        
        data = data.to(device=DEVICE)
        targets = targets.float().unsqueeze(1).to(device=DEVICE)
        
        # forward
        with torch.cuda.amp.autocast():
            predictions = model(data)
            loss = loss_fn(predictions, targets)
        
        # backward
        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        # update tqdm loop (showing loss function thus far)
        loop.set_postfix(loss=loss.item())
        
        # ^^--- trains one epoch ---

### Train Model

In [ ]:
# MAIN FUNCTION:
train_transform = A.Compose(
    [
        A.Resize(height=IMAGE_HEIGHT, width=IMAGE_WIDTH),
        A.Rotate(limit=35, p=1.0),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.1),
        A.Normalize(
            mean=[0.0, 0.0, 0.0],
            std=[1.0, 1.0, 1.0],
            max_pixel_value=255.0,
        ),
        ToTensorV2(),
    ],
)

val_transforms = A.Compose(
    [
        A.Resize(height=IMAGE_HEIGHT, width=IMAGE_WIDTH),
        A.Normalize(
            mean=[0.0, 0.0, 0.0],
            std=[1.0, 1.0, 1.0],
            max_pixel_value=255.0
        ),
        ToTensorV2()
    ]
)

class Paths:
    data_root = '/kaggle/input/google-research-identify-contrails-reduce-global-warming'
    test_data_root = '/kaggle/input/google-research-identify-contrails-reduce-global-warming/test'
    contrails = '/kaggle/input/contrails-images-ash-color/contrails/'
    train_path = '/kaggle/input/contrails-images-ash-color/train_df.csv'
    valid_path = '/kaggle/input/contrails-images-ash-color/valid_df.csv'

# Get dataframe
train_df = pd.read_csv(Paths.train_path)
valid_df = pd.read_csv(Paths.valid_path)

train_df['path'] = Paths.contrails + train_df['record_id'].astype(str) + '.npy'
valid_df['path'] = Paths.contrails + valid_df['record_id'].astype(str) + '.npy'

# For multi-class segmentation, change "out_channels" to however many classes, and change BCEWithLogitsLoss to cross entropy loss
model = UNET(in_channels=3, out_channels=1).to(DEVICE)
loss_fn = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

# Gets the pytorch.DataLoader objects (i.e. the datasets except they're iterable)
train_loader, val_loader = get_loaders(
    train_df,
    valid_df,
    BATCH_SIZE,
    NUM_WORKERS,
    PIN_MEMORY
)

# if LOAD_MODEL:
#     load_checkpoint(torch.load("my_checkpoint.pth.tar"), model)

#     # Resaves predictions in folder
#     save_predictions_as_imgs(
#         val_loader, model, folder=os.path.join(ROOT2, "saved_images"), device=DEVICE
#     )

#     # Do this after load model
#     check_accuracy(val_loader, model, device=DEVICE)

scaler = torch.cuda.amp.GradScaler()

for epoch in range(NUM_EPOCHS):

    print(f"Training (epoch {epoch})")
    train_fn(train_loader, model, optimizer, loss_fn, scaler)

    print("Saving model")
    # save model
    checkpoint = {
        "state_dict": model.state_dict(),
        "optimizer":optimizer.state_dict()
    }
    save_checkpoint(checkpoint)

    print("Check accuracy")
    # check accuracy
    check_accuracy(val_loader, model, device=DEVICE)

    # print some examples to a folder
    # save_predictions_as_imgs(
    #     val_loader, model, folder="saved_images/", device=DEVICE
    # )

    print("Saving predictions as image")

    # Note: Only predicted images for validation images are saved. That's why they're so few.
#     save_predictions_as_imgs(
#         val_loader, model, folder=os.path.join(ROOT2, "saved_images"), device=DEVICE
#     )

/opt/conda/lib/python3.10/site-packages/torch/cuda/amp/grad_scaler.py:120: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")


Training (epoch 0)


  0%|          | 0/1284 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/torch/amp/autocast_mode.py:204: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
/opt/conda/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnin

# Class for test dataset

In [ ]:
class ContrailsTestDataset(torch.utils.data.Dataset):
    def __init__(self, df, train=True):
        
        self.df = df
        self.trn = train
    
    # Handles reading from a directory under test (i.e. reads band files)
    # Note: Only bands 11, 14, and 15 are used to create the final tensor
    def read_record(self, directory):
        
        # Stores numpy arrays in a dictionary
        record_data = {}
        for x in [
            "band_11", 
            "band_14", 
            "band_15"
        ]:

            record_data[x] = np.load(os.path.join(directory, x + ".npy"))

        # Returns dictionary mapping band name (i.e. "band_11") to numpy array for that band file
        # (i.e. band_11.npy converted to numpy array)
        return record_data

    def normalize_range(self, data, bounds):
        """Maps data to the range [0, 1]."""
        return (data - bounds[0]) / (bounds[1] - bounds[0])
    
    # This is the function responsible for taking multiple bands as input, and returning a single tensor
    # "False color" = ": color in an image (such as a photograph) of an object that does not actually 
    # appear in the object but is used to enhance, contrast, or distinguish details."
    def get_false_color(self, record_data):
        _T11_BOUNDS = (243, 303)
        _CLOUD_TOP_TDIFF_BOUNDS = (-4, 5)
        _TDIFF_BOUNDS = (-4, 2)
        
        N_TIMES_BEFORE = 4
        
        # "Combines" the 'band_15', 'band_14', and 'band_11' tensors (each are 3D, H x W x Time step)
        # Note: r, g, and b are still H x W x Time step (256 x 256 x 8)
        r = self.normalize_range(record_data["band_15"] - record_data["band_14"], _TDIFF_BOUNDS)
        g = self.normalize_range(record_data["band_14"] - record_data["band_11"], _CLOUD_TOP_TDIFF_BOUNDS)
        b = self.normalize_range(record_data["band_14"], _T11_BOUNDS)
        
        display('rgb:')
        display(len(r))
        display(len(r[0]))
        display(len(r[0][0]))
        display(g)
        display(b)
        
        # Combines 3 x (H x W x T or 256 x 256 x 8) tensors into one H x W x C x T (256 x 256 x 3 x 8)
        false_color = np.clip(np.stack([r, g, b], axis=2), 0, 1)
        display('false_color')
        display(false_color.shape)
        
        # Slice includes all dimensions 1 - 3, but only the 'N_TIMES_BEFORE'th 4th dimension element
        img = false_color[..., N_TIMES_BEFORE]

        return img
    
    def __getitem__(self, index):
        row = self.df.iloc[index]
        con_path = row.path
        data = self.read_record(con_path)    
        
        display('getting data')
#         display(data)
        display('band_11')
        display(len(data['band_11']))
        display(len(data['band_11'][0]))
        display(len(data['band_11'][0][0]))
        img = self.get_false_color(data)
        
        display('img:')
        display(len(img))
        display(len(img[0]))
        display(len(img[0][0]))
        display(img)
        
        img = torch.tensor(img)
        
        display('converted to tensor')
        display(img)
        
        # Changes tensor shape from H x W x C (256 x 256 x 3) to C x H x W (3 x 256 x 256)
        img = img.permute(2, 0, 1)
        
        display('ret img')
        display(img.shape)
        
        return img.float()
    
    def __len__(self):
        return len(self.df)

# Get dataset

In [ ]:
filenames = os.listdir(Paths.test_data_root)
test_df = pd.DataFrame(filenames, columns=['record_id'])

test_df['path'] = Paths.test_data_root + test_df['record_id'].astype(str)


test_ds = ContrailsTestDataset(
        test_df,
        train = False
    )

display(len(test_ds))
display(test_ds)
display(len(test_ds[0]))
display(test_ds[0])

test_dl = DataLoader(test_ds, batch_size=Config.batch_size, num_workers = 2)

# Inference
Generates predictions for test data here

In [ ]:
test_preds = []

progress_bar = tqdm(test_dl)

for i, x in enumerate(progress_bar)
    with torch.cuda.amp.autocast():
        pred = model(data)
        test_preds.append(pred)

# Convert to RLE format
Courtesy of Inversion :)

In [ ]:
def rle_encode(x, fg_val=1):
    """
    Args:
        x:  numpy array of shape (height, width), 1 - mask, 0 - background
    Returns: run length encoding as list
    """

    dots = np.where(
        x.T.flatten() == fg_val)[0]  # .T sets Fortran order down-then-right
    run_lengths = []
    prev = -2
    for b in dots:
        if b > prev + 1:
            run_lengths.extend((b + 1, 0))
        run_lengths[-1] += 1
        prev = b
    return run_lengths


def list_to_string(x):
    """
    Converts list to a string representation
    Empty list returns '-'
    """
    if x: # non-empty list
        s = str(x).replace("[", "").replace("]", "").replace(",", "")
    else:
        s = '-'
    return s

# Submission

In [ ]:
test_preds = [list_to_string(rle_encode(i)) for i in test_preds]

test_ids = os.listdir(Paths.test_dataset_root)

submission = pd.read_csv(pd.DataFrame('record_id':test_ids, 'encoded_pixels':test_preds))

submission.head()

submission.to_csv('submission.csv')